# Example of initializing an instrument with pyvisa in QCoDeS over a USB interface

Based on the following [QCoDeS example documentation](https://qcodes.github.io/Qcodes/examples/driver_examples/Qcodes%20example%20with%20Yokogawa%20GS2xx.html), expect that a different method is used to connect to the instrument. Simply initializing the ni-visa backend did not work for some reason, so first the pyvisa-py backend opens a connection with the interface, followed by the ni-visa backend. Only then is the instrument properly recognized and can connect.

In [1]:
#from gpib_ctypes import make_default_gpib
#make_default_gpib()

# importing packages
import numpy as np
import time
import qcodes
import pyvisa as visa
import ipywidgets as widgets
# enable to get a more debuggable output:
#visa.log_to_screen()

# pyvisa-py resource manager
py_rm = visa.ResourceManager('@py')
# national instruments resource manager
ni_rm = visa.ResourceManager()


print(py_rm)
print(ni_rm) 

Resource Manager of Visa Library at py
Resource Manager of Visa Library at /usr/lib/x86_64-linux-gnu/libvisa.so.21.5.0


Select the one that you are interested in testing the connection, choose `USB` over `ASRL` for a USB device to work properly

In [2]:
# list the resources that the ni-visa backend detects, should look something like:
# `('USB0::0x0B21::0x0039::91LA25023::INSTR', 'ASRL5::INSTR')`
print("ni-visa: " + str(ni_rm.list_resources()))

ni-visa: ('ASRL1::INSTR', 'ASRL2::INSTR', 'ASRL3::INSTR', 'ASRL4::INSTR', 'ASRL5::INSTR', 'ASRL6::INSTR', 'ASRL7::INSTR', 'ASRL8::INSTR', 'ASRL9::INSTR', 'ASRL10::INSTR', 'ASRL11::INSTR', 'ASRL12::INSTR', 'ASRL13::INSTR', 'ASRL14::INSTR', 'ASRL15::INSTR', 'ASRL16::INSTR', 'ASRL17::INSTR', 'ASRL18::INSTR', 'ASRL19::INSTR', 'ASRL20::INSTR', 'ASRL21::INSTR', 'ASRL22::INSTR', 'ASRL23::INSTR', 'ASRL24::INSTR', 'ASRL25::INSTR', 'ASRL26::INSTR', 'ASRL27::INSTR', 'ASRL28::INSTR', 'ASRL29::INSTR', 'ASRL30::INSTR', 'ASRL31::INSTR', 'ASRL32::INSTR')


Select the one that you are interested in testing the connection, choose `USB` over `ASRL` for a USB device to work properly

In [4]:
### IMPORTANT ###
# for some reason, the USB interface of the instrument has to be opened with the pyvisa-py resource manager before ni-visa

# so, open whichever interface was selected in the prior cell
#py_rm.open_resource(py_rm_interface.value)

# show the interfaces that the pyvisa-py backend detects
ni_rm_interface = widgets.Dropdown(
    options=ni_rm.list_resources(),
    #value='2',
    description='ni Interface:',
    disabled=False,
    layout={'width': 'max-content'}
)

display(ni_rm_interface)

Dropdown(description='ni Interface:', layout=Layout(width='max-content'), options=('ASRL1::INSTR', 'ASRL2::INS…

Here is an example of how you would connect to a Yokogama GS210 with integrated QCoDeS drivers:

In [5]:
# importing an instrument driver
from qcodes.instrument_drivers.yokogawa.GS200 import GS200

# intialize the instrument with the following parameters

# the instrument is initialized and stored as a variable gs, where "gs200" is its name
gs = GS200("gs200", address=ni_rm_interface.value, terminator="\n")

# close the pyvisa-py resource manager:
py_rm.close()

[gs200(GS200)] Could not connect at ASRL19::INSTR
Traceback (most recent call last):
  File "/home/nikolai/miniconda3/envs/qcodes/lib/python3.9/site-packages/qcodes/instrument/visa.py", line 79, in __init__
    self.set_address(address)
  File "/home/nikolai/miniconda3/envs/qcodes/lib/python3.9/site-packages/qcodes/instrument/visa.py", line 118, in set_address
    resource = resource_manager.open_resource(address)
  File "/home/nikolai/miniconda3/envs/qcodes/lib/python3.9/site-packages/pyvisa/highlevel.py", line 3304, in open_resource
    res.open(access_mode, open_timeout)
  File "/home/nikolai/miniconda3/envs/qcodes/lib/python3.9/site-packages/pyvisa/resources/resource.py", line 297, in open
    self.session, status = self._resource_manager.open_bare_resource(
  File "/home/nikolai/miniconda3/envs/qcodes/lib/python3.9/site-packages/pyvisa/highlevel.py", line 3232, in open_bare_resource
    return self.visalib.open(self.session, resource_name, access_mode, open_timeout)
  File "/home/

VisaIOError: VI_ERROR_NPERMISSION (-1073807192): Access to the resource or remote machine is denied. This is due to lack of sufficient privileges for the current user or machine